# 推薦システムの評価

## 評価のタイミング

- オフライン評価：デプロイ前に、過去のログデータを用いて性能を評価する方法
- オンライン評価：デプロイ後にA/Bテストで新しい推薦モデルの性能を評価する方法

## データの分割方法

オフライン評価の際に、データセットをどう分割するか

- 無作為分割：普通のホールドアウト法。推薦システムには適さない可能性がある。
- 時間ベースの分割：ある時点を堺にtrain/testを分ける。将来データへの性能を評価したい場合に有効
- ユーザーベースの分割：新規ユーザーへの性能を評価したい場合に有効
- アイテムベースの分割：新規アイテムへの性能を評価したい場合に有効

## 精度評価指標

一般的な機械学習のように予測精度を評価する場合に使われる指標

### レイティング予測の指標

- RMSE
- MAE

### 確率予測の指標

（クリック率予測など）

- 対数尤度: 

$$
\begin{aligned}
\text { Log-likelihood } & =\sum_{(i, j) \in \boldsymbol{\Omega}^{\mathrm{test}}} \log \operatorname{Pr}\left(y_{i j} \mid \hat{p}_{i j}\right) \\
& =\sum_{(i, j) \in \boldsymbol{\Omega}^{\mathrm{test}}} y_{i j} \log \left(\hat{p}_{i j}\right)+\left(1-y_{i j}\right) \log \left(1-\hat{p}_{i j}\right)
\end{aligned}
$$


:::{admonition} 精度評価指標の限界
:class: warning

次のような限界があるため、精度評価指標に依存してはならない

**ランキングの測定に不適**

- 多くのシステムではランキング上位のアイテムしか推薦されないため、高順位になる可能性が高いアイテムの精度の重要度が低順位のものよりも高い

**精度の改善とビジネス上のKPIとの関連性の低さ**

- 予測精度の改善がコンバージョン率などの改善につながるとは限らず、精度評価指標をKPI改善に役立てるのが困難
:::

## ランキング指標

### 大局的ランキング指標

**大局的ランキング指標（global ranking metrics）**は、モデルによる推定スコアでテストセットを順序付けしたとき、評価の高いユーザー・アイテム対が低い評価の対より上位に順位付けされる度合いを測るもの。


#### 二値分類の場合

二値分類（例：クリック予測）や二値化されうる問題設定の場合、PR曲線やROC曲線を使った評価ができる

#### それ以外の場合

他の方法としては順位相関がある


### 局所的ランキング指標

**局所的ランキング指標（local ranking metrics）**は、各ユーザーについてアイテムを個別に順位付けする方法。高評価アイテムが低評価アイテムよりも上位に順位付けされているかどうかを測定し、その後ユーザー間で平均を取る。

#### 二値分類の場合

##### 順位Kでの適合率（P@K）

各ユーザーについて、予測したスコアでアイテムを降順に順位付けする。

上位K個のアイテムのうちの性のアイテムの部分集合

##### 平均適合率の平均（mAP: mean average precision）

P@Kの平均が平均適合率になり、それをユーザーについて平均したものがmAPになる

##### 正規化割引累積利得（nDCG: normalized discounted cumulative gain）

順位$k$のアイテムにユーザー$i$が正のレイティングを与えた場合$p_i(k)=1$で、そうでなければ$p_i(k)=0$とする。

テストセットにおけるユーザー$i$によって評価されたアイテム集合$\mathcal{J}_i^{test}$の要素数を$n_i$としたとき

割引累積利得（DCG）を次のように定義する

$$
DCG_i = p_i(1) + \sum^{n_i}_{k=2} \frac{p_i(k)}{\log_2 k}
$$

nDCGはユーザー$i$の最大DCGで正規化したDCGである

$$
nDCG_i = \frac{DCG_i}{ 1 + \sum^{n_i^+}_{k=2} \frac{1}{\log_2 k} }
$$

ここで$n_i^+$は$\mathcal{J}_i^{test}$におけるユーザー$i$が正のレイティングをしたアイテム数


#### それ以外の場合

二値分類でない場合や二値分類に変換できない場合、平均順位相関を使用する


